# Kaggle: SF Crime 

In [2]:
# %matplotlib qt

In [3]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn import preprocessing
# from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from os.path import expanduser, normpath
import time
import datetime

In [4]:
# Set paths for data to be imported

home = expanduser('~')
# path = str(home) + '\\Documents\\data-science\\kaggle\\sf-crime\\' # Windows
path = str(home) + '/Documents/Personal/data-science/kaggle/sf-crime/' # Mac
trainfile = 'train.csv'
testfile = 'test.csv'

In [5]:
train_data = pd.read_csv(path+trainfile)
test_data = pd.read_csv(path+testfile)

In [ ]:
# train_data.groupby('Category').size().sort_values()

## Features

In [6]:
t0 = time.time()

train_data['Dates'] = pd.to_datetime(train_data['Dates'])
train_data['year'] = train_data['Dates'].dt.year
train_data['month'] = train_data['Dates'].dt.month 
train_data['day'] = train_data['Dates'].dt.day
train_data['hour'] = train_data['Dates'].dt.hour
train_data['minute'] = train_data['Dates'].dt.minute

t1 = time.time()
print(t1-t0)

0.6993160247802734


In [7]:
t0 = time.time()

test_data['Dates'] = pd.to_datetime(test_data['Dates'])
test_data['year'] = test_data['Dates'].dt.year
test_data['month'] = test_data['Dates'].dt.month 
test_data['day'] = test_data['Dates'].dt.day
test_data['hour'] = test_data['Dates'].dt.hour
test_data['minute'] = test_data['Dates'].dt.minute

t1 = time.time()
print(t1-t0)

In [8]:
# Set up 0 to 1 scaler for preprocessing features in training and test sets
min_max_scaler = preprocessing.MinMaxScaler()

In [14]:
train_data_minmax = pd.DataFrame(min_max_scaler.fit_transform(train_data),
                                 index = train_data.index, 
                                 columns = train_data.columns) 

In [ ]:
test_data_minmax = pd.DataFrame(min_max_scaler.transform(test),
                                index = test_data.index, 
                                columns = test_data.columns) 

In [9]:
# generate training and cross-validation features
X_train, X_cv, y_train, y_cv = cross_validation.train_test_split(train_data.ix[:,['year','month','day','hour','minute','X','Y']], train_data.ix[:,'Category'], test_size=0.80, random_state=0)

In [ ]:
X_test = test_data.ix[:,['year','month','day','hour','minute','X','Y']]

## Regression Model

In [ ]:
# def MAPE(true, predicted):
#     return np.mean(np.abs((true - predicted)/true))

# def RMSE(error):
#     return (np.mean(error**2))**.05

In [ ]:
# SUPPORT VECTOR MACHINE
t0 = time.time()

crime_svm = LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
crime_svm.fit(X_train, y_train)

y_train_pred = crime_svm.predict(X_train)
y_cv_pred = crime_svm.predict(X_cv)

t1 = time.time()
total = t1-t0
print (total) # 19 minutes

In [ ]:
# SUPPORT VECTOR MACHINE
t0 = time.time()

crime_svm = OneVsRestClassifier(SVC(kernel='rbf'))
crime_svm.fit(X_train, y_train)

y_train_pred = crime_svm.predict(X_train)
y_cv_pred = crime_svm.predict(X_cv)

t1 = time.time()
total = t1-t0
print (total)

In [ ]:
t0 = time.time()

score_train = crime_svm.score(X_train, y_train)

t1 = time.time()
total = t1-t0
print ('Score:', score_train, ', Time:', total) # Score:  0.0880649625019 , Time:  0.27322959899902344

In [ ]:
t0 = time.time()

score_cv = crime_svm.score(X_cv, y_cv)

t1 = time.time()
total = t1-t0
print ('Score:', score_cv, ', Time:', total) # Score:  0.0874252605205 , Time:  1.2154865264892578

In [ ]:
t0 = time.time()

output = crime_svm.predict(X_test)

t1 = time.time()
total = t1-t0
print ('Time:', total) # Time: 0.3079204559326172

In [ ]:
pd.get_dummies(output)

In [ ]:
cat_uniques = train_data.Category.unique()

In [ ]:
headers = 'Id,' + ','.join(sorted(cat_uniques)) + '\n'
f = open('y_test.csv', 'w')
f.write(headers)
for i in xrange(len(y_test)):
    arr = [0] * 39
    arr[int(y_test)]